## 00 Creación de tabla de exportación

In [0]:
%sql
CREATE TABLE databricks_clase.prueba_schema.clientes_campana(
  col1 INT,
  col2 STRING
)
USING DELTA

## Lectura de bases

In [0]:
base_clientes = spark.table('databricks_clase.prueba_schema.base_clientes')
base_prediccion = spark.table('databricks_clase.prueba_schema.base_prediccion')

In [0]:
base_clientes.show()

+-----+----+-----------+------------+-------------------+--------------+--------+--------+-------------+------------------+-------+-----------------+-----------------+-------------+-----------+-----------+---------+
|   ID|EDAD|    TRABAJO|ESTADO_CIVIL|          EDUCACION|INCUMPLIMIENTO|VIVIENDA|PRESTAMO|TIPO_CONTACTO|PERIODO_AFILIACION|CAMPANA|NRO_DIAS_CONTACTO|CONTACTOS_PREVIOS|RESULT_PREVIO|    INGRESO|MONTO_DEUDA|  Celular|
+-----+----+-----------+------------+-------------------+--------------+--------+--------+-------------+------------------+-------+-----------------+-----------------+-------------+-----------+-----------+---------+
|27692|  38|     admin.|      single|  university.degree|            no|     yes|      no|     cellular|            202403|      2|              999|                0|  nonexistent|5265.038074|7281.213884|940823824|
|27693|  26|    student|      single|           basic.9y|            no|      no|      no|     cellular|            202403|      4|     

In [0]:
base_prediccion.show()

+---+-----------------+----------+
| ID|PERIODO_MARCACION|ACEPTACION|
+---+-----------------+----------+
|  1|           202501|         0|
|  2|           202501|         0|
|  3|           202501|         0|
|  4|           202501|         0|
|  5|           202501|         0|
|  6|           202501|         0|
|  7|           202501|         0|
|  8|           202501|         0|
|  9|           202501|         0|
| 11|           202501|         0|
| 13|           202501|         0|
| 15|           202501|         0|
| 16|           202501|         0|
| 18|           202501|         0|
| 21|           202501|         0|
| 24|           202501|         0|
| 25|           202501|         0|
| 28|           202501|         0|
| 29|           202501|         0|
| 31|           202501|         0|
+---+-----------------+----------+
only showing top 20 rows


## Exploración de los datos

In [0]:
base_clientes_pandas = base_clientes.toPandas()
base_prediccion_pandas = base_prediccion.toPandas()

In [0]:
base_clientes_pandas.describe()

,ID,EDAD,PERIODO_AFILIACION,CAMPANA,NRO_DIAS_CONTACTO,CONTACTOS_PREVIOS,INGRESO,MONTO_DEUDA,Celular
count,29441.000000,29441.000000,29441.000000,29441.000000,29441.000000,29441.000000,29441.000000,29441.000000,2.944100e+04
mean,20566.007099,40.392616,202407.181923,2.559696,962.502666,0.174926,3917.532231,19952.445440,1.222383e+09
std,11907.861668,12.545372,2.447448,2.751564,186.854468,0.497860,3936.598033,8403.697791,1.503708e+09
min,1.000000,5.000000,202403.000000,1.000000,0.000000,0.000000,1125.000000,6346.668502,9.000000e+08
25%,10187.000000,32.000000,202405.000000,1.000000,999.000000,0.000000,1323.738750,12260.656290,9.257158e+08
50%,20604.000000,38.000000,202406.000000,2.000000,999.000000,0.000000,1495.621568,19869.674050,9.516263e+08
75%,30856.000000,47.000000,202410.000000,3.000000,999.000000,0.000000,7211.614999,27639.347630,9.776477e+08
max,41188.000000,160.000000,202412.000000,56.000000,999.000000,6.000000,16463.000000,33653.594950,9.998266e+09


In [0]:
base_prediccion_pandas.describe()

,ID,PERIODO_MARCACION,ACEPTACION
count,29441.000000,29441.0,29441.000000
mean,20566.007099,202501.0,0.112190
std,11907.861668,0.0,0.315606
min,1.000000,202501.0,0.000000
25%,10187.000000,202501.0,0.000000
50%,20604.000000,202501.0,0.000000
75%,30856.000000,202501.0,0.000000
max,41188.000000,202501.0,1.000000


In [0]:
base_clientes_pandas.describe(include="object")

,TRABAJO,ESTADO_CIVIL,EDUCACION,INCUMPLIMIENTO,VIVIENDA,PRESTAMO,TIPO_CONTACTO,RESULT_PREVIO
count,29441,29441,29441,29441,29441,29441,29441,29441
unique,12,10,8,3,3,3,1,3
top,admin.,married,university.degree,no,yes,no,cellular,nonexistent
freq,7457,17713,8659,23314,15456,24322,29441,25383


In [0]:
for i in ['TRABAJO', 'ESTADO_CIVIL', 'EDUCACION', 'INCUMPLIMIENTO', 'VIVIENDA', 'PRESTAMO', 'TIPO_CONTACTO', 'RESULT_PREVIO']:
    print(base_clientes_pandas[i].value_counts())

admin.           7457
blue-collar      6623
technician       4824
services         2819
management       2079
retired          1247
self-employed    1038
entrepreneur     1029
housemaid         753
unemployed        723
student           609
unknown           240
Name: TRABAJO, dtype: int64
married      17713
single        8185
divorced      3245
 married       118
unknown         58
singgle         54
marrid          30
divorced        23
single          10
d_ivorced        5
Name: ESTADO_CIVIL, dtype: int64
university.degree      8659
high.school            6757
basic.9y               4362
professional.course    3798
basic.4y               2972
basic.6y               1642
unknown                1235
illiterate               16
Name: EDUCACION, dtype: int64
no         23314
unknown     6124
yes            3
Name: INCUMPLIMIENTO, dtype: int64
yes        15456
no         13293
unknown      692
Name: VIVIENDA, dtype: int64
no         24322
yes         4427
unknown      692
Name: PRESTAMO

In [0]:
base_clientes_pandas.ESTADO_CIVIL.unique()

array(['single', 'married', 'divorced', ' married', 'singgle ', 'unknown',
       'single ', 'marrid', 'divorced ', 'd_ivorced'], dtype=object)

## Limpieza de los datos

In [0]:
from pyspark.sql.functions import col, when

In [0]:
# Eliminamos la columna "TIPO_CONTACTO" porque no es informativa:

base_clientes = base_clientes.drop('TIPO_CONTACTO')

# Cambiamos las etiquetas de la variable "ESTADO_CIVIL" por error en calidad de datos:

base_clientes = base_clientes.withColumn(
    "ESTADO_CIVIL",
    when(col("ESTADO_CIVIL") == "singgle ", "single")
    .when(col("ESTADO_CIVIL") == " married", "married")
    .when(col("ESTADO_CIVIL") == "single ", "single")
    .when(col("ESTADO_CIVIL") == "d_ivorced", "divorced")
    .when(col("ESTADO_CIVIL") == "divorced ", "divorced")
    .when(col("ESTADO_CIVIL") == "marrid", "married")
    .otherwise(col("ESTADO_CIVIL"))
)


## Exportación de las tablas

In [0]:
base_clientes.write.mode("overwrite").saveAsTable('databricks_clase.prueba_schema.base_clientes')

In [0]:
# Vemos el historial de la tabla

from delta.tables import DeltaTable
base_clientes_delta = DeltaTable.forName(spark, "databricks_clase.prueba_schema.base_clientes")

In [0]:
base_clientes_delta.history().show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      1|2025-03-03 03:08:05|8926564628570346|ursula.cotrina27@...|CREATE OR REPLACE...|{partitionBy -> [...|NULL|{3543531657336052}|0207-032915-4307u1zf|          0|WriteSerializable|        false|{numFiles -> 1, n...|        NULL|Databr